#Chapter 2 - Introduction to Spark

In [ ]:
myRange = spark.range(1000).toDF("number")


In [ ]:
divisBy2 = myRange.where("number % 2 = 0")


In [ ]:
divisBy2 = myRange.where("number % 2 = 0").show()

+------+
number|
+------+
 0|
 2|
 4|
 6|
 8|
 10|
 12|
 14|
 16|
 18|
 20|
 22|
 24|
 26|
 28|
 30|
 32|
 34|
 36|
 38|
+------+
only showing top 20 rows

In [ ]:
divisBy2.count()


Out[22]: 500

In [ ]:
flightData2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("dbfs:/FileStore/2015_summary.csv")

In [ ]:
flightData2015.take(3)

Out[24]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [ ]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#425 ASC NULLS FIRST], true, 0
 +- Exchange rangepartitioning(count#425 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#552]
 +- FileScan csv [DEST_COUNTRY_NAME#423,ORIGIN_COUNTRY_NAME#424,count#425] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [ ]:
flightData2015.sort("count").take(2)

Out[27]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [ ]:
#make any DataFrame into a table or view with one simple method call:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [ ]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

sqlWay.explain()
dataFrameWay.explain()



== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#423], functions=[finalmerge_count(merge count#451L) AS count(1)#439L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#423, 5), ENSURE_REQUIREMENTS, [id=#593]
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#423], functions=[partial_count(1) AS count#451L])
 +- FileScan csv [DEST_COUNTRY_NAME#423] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#423], functions=[finalmerge_count(merge count#456L) AS count(1)#446L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#423, 5), ENSURE_REQUIREMENTS, [id=#639]
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#423], functions=[partial_count(1) AS count#456L])
 +- FileScan csv [DEST_COUNTRY_NAME#423] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [ ]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

Out[30]: [Row(max(count)=370002)]

In [ ]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

Out[33]: [Row(max(count)=370002)]

In [ ]:
#the top five destination countries in the data.Multitransformation query

maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [ ]:
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [ ]:
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#560L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#423,destination_total#560L])
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#423], functions=[finalmerge_sum(merge sum#564L) AS sum(count#425)#556L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#423, 5), ENSURE_REQUIREMENTS, [id=#1049]
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#423], functions=[partial_sum(count#425) AS sum#564L])
 +- FileScan csv [DEST_COUNTRY_NAME#423,count#425] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>

#Chapter 4 - Structured API Overview

In [ ]:
df = spark.range(500).toDF("number")
df.select(df["number"] + 10)

Out[37]: DataFrame[(number + 10): bigint]

In [ ]:
#rows
spark.range(2).collect()

Out[38]: [Row(id=0), Row(id=1)]

In [ ]:
from pyspark.sql.types import *
b = ByteType()

# Chapter 5 - Basic Structured Operations

In [ ]:
df = spark.read.format("json").load("dbfs:/FileStore/2015_summary.json")

In [ ]:
df.printSchema()

root
-- DEST_COUNTRY_NAME: string (nullable = true)
-- ORIGIN_COUNTRY_NAME: string (nullable = true)
-- count: long (nullable = true)

## Schemas

In [ ]:
spark.read.format("json").load("dbfs:/FileStore/2015_summary.json").schema

Out[43]: StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [ ]:
#myManualSchema
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
StructField("DEST_COUNTRY_NAME", StringType(), True),
StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
StructField("count", LongType(), False, metadata={"hello":"world"})])

df = spark.read.format("json").schema(myManualSchema)\
.load("dbfs:/FileStore/2015_summary.json")

## Columns and Expressions

In [ ]:
from pyspark.sql.functions import col, column
col("someColumnName")
column("someColumnName")

Out[47]: Column<'someColumnName'>

In [ ]:
spark.read.format("json").load("dbfs:/FileStore/2015_summary.json").columns

Out[52]: ['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## Expressions

In [ ]:
(((col("someCol") + 5) * 200) - 6) < col("otherCol")

Out[48]: Column<'((((someCol + 5) * 200) - 6) < otherCol)'>

In [ ]:
from pyspark.sql.functions import expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

Out[49]: Column<'((((someCol + 5) * 200) - 6) < otherCol)'>

## Records and Rows

In [ ]:
df.first()

Out[53]: Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [ ]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

In [ ]:

print(myRow[0])
myRow[2]


Hello
Out[58]: 1

## DataFrame Transformations

In [ ]:
df = spark.read.format("json").load("dbfs:/FileStore/2015_summary.json")
df.createOrReplaceTempView("dfTable")

In [ ]:
#my Df
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
myManualSchema = StructType([
StructField("some", StringType(), True),
StructField("col", StringType(), True),
StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
 some| col|names|
+-----+----+-----+
Hello|null| 1|
+-----+----+-----+

## select and selectExpr

In [ ]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
DEST_COUNTRY_NAME|
+-----------------+
 United States|
 United States|
+-----------------+
only showing top 2 rows

In [ ]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
 United States| Romania|
 United States| Croatia|
+-----------------+-------------------+
only showing top 2 rows

In [ ]:
from pyspark.sql.functions import expr, col, column
df.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)


+-----------------+-----------------+-----------------+
DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
 United States| United States| United States|
 United States| United States| United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows

In [ ]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
 destination|
+-------------+
United States|
United States|
+-------------+
only showing top 2 rows

In [ ]:
df.select(expr("DEST_COUNTRY_NAME as destination"))\
.show(2)

+-------------+
 destination|
+-------------+
United States|
United States|
+-------------+
only showing top 2 rows

In [ ]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+
DEST_COUNTRY_NAME|
+-----------------+
 United States|
 United States|
+-----------------+
only showing top 2 rows

In [ ]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
United States| United States|
United States| United States|
+-------------+-----------------+
only showing top 2 rows

In [ ]:
df.selectExpr(
"*", 
"(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(2)

+-----------------+-------------------+-----+-------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
 United States| Romania| 15| false|
 United States| Croatia| 1| false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows

In [ ]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
 avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
1770.765625| 132|
+-----------+---------------------------------+

In [ ]:
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
 United States| Romania| 15| 1|
 United States| Croatia| 1| 1|
+-----------------+-------------------+-----+---+
only showing top 2 rows

## Adding Columns

In [ ]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
 United States| Romania| 15| 1|
 United States| Croatia| 1| 1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows

In [ ]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-------------------+-----+-------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
 United States| Romania| 15| false|
 United States| Croatia| 1| false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows

In [ ]:
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

Out[80]: ['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

## Renaming Columns

In [ ]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

Out[83]: ['dest', 'ORIGIN_COUNTRY_NAME', 'count']

## Reserved Characters and Keywords

In [ ]:
dfWithLongColName = df.withColumn(
"This Long Column-Name",
expr("ORIGIN_COUNTRY_NAME"))

In [ ]:
dfWithLongColName.selectExpr(
"`This Long Column-Name`",
"`This Long Column-Name` as `new col`")\
.show(2)
dfWithLongColName.createOrReplaceTempView("dfTableLong")

+---------------------+-------+
This Long Column-Name|new col|
+---------------------+-------+
 Romania|Romania|
 Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows

In [ ]:
dfWithLongColName.select(expr("`This Long Column-Name`")).columns

Out[86]: ['This Long Column-Name']

## Removing Columns

In [ ]:
df.drop("ORIGIN_COUNTRY_NAME").columns

Out[88]: ['DEST_COUNTRY_NAME', 'count']

In [ ]:
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME")

Out[89]: DataFrame[count: bigint, This Long Column-Name: string]

## Changing a Column’s Type (cast)

In [ ]:
df.withColumn("count2", col("count").cast("long"))

Out[90]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: bigint]

## Filtering Rows

In [ ]:
df.filter(col("count") < 2).show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Croatia| 1|
 United States| Singapore| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

In [ ]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Croatia| 1|
 United States| Singapore| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

In [ ]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
.show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Singapore| 1|
 Moldova| United States| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

## Getting Unique Rows

In [ ]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

Out[94]: 256

In [ ]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

Out[95]: 125

## Random Samples

In [ ]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

Out[96]: 138

## Random Splits

In [ ]:
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count() # False

Out[97]: False

## Concatenating and Appending Rows (Union)

In [ ]:
from pyspark.sql import Row
schema = df.schema
newRows =[Row("New Country", "Other Country", 5), Row("New Country 2", "Other Country 3", 1)]

In [ ]:
parallelizedRows = spark.sparkContext.parallelize(newRows)


In [ ]:
newDF = spark.createDataFrame(parallelizedRows, schema)

df.union(newDF)\
.where("count = 1")\
.where(col("ORIGIN_COUNTRY_NAME") != "United States")\
.show()

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Croatia| 1|
 United States| Singapore| 1|
 United States| Gibraltar| 1|
 United States| Cyprus| 1|
 United States| Estonia| 1|
 United States| Lithuania| 1|
 United States| Bulgaria| 1|
 United States| Georgia| 1|
 United States| Bahrain| 1|
 United States| Papua New Guinea| 1|
 United States| Montenegro| 1|
 United States| Namibia| 1|
 New Country 2| Other Country 3| 1|
+-----------------+-------------------+-----+

## Sorting Rows

In [ ]:
df.sort("count").show(5)


+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 Malta| United States| 1|
Saint Vincent and...| United States| 1|
 United States| Croatia| 1|
 United States| Gibraltar| 1|
 United States| Singapore| 1|
+--------------------+-------------------+-----+
only showing top 5 rows

In [ ]:
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 Burkina Faso| United States| 1|
 Cote d'Ivoire| United States| 1|
 Cyprus| United States| 1|
 Djibouti| United States| 1|
 Indonesia| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

In [ ]:
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)


+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 Burkina Faso| United States| 1|
 Cote d'Ivoire| United States| 1|
 Cyprus| United States| 1|
 Djibouti| United States| 1|
 Indonesia| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

In [ ]:
from pyspark.sql.functions import desc, asc
df.orderBy(expr("count desc")).show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 Moldova| United States| 1|
 United States| Croatia| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

In [ ]:
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
 United States| United States|370002|
 United States| Canada| 8483|
+-----------------+-------------------+------+
only showing top 2 rows

## Limit

In [ ]:
df.limit(5).show()

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
+-----------------+-------------------+-----+

In [ ]:
df.orderBy(expr("count desc")).limit(6).show()

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 Malta| United States| 1|
Saint Vincent and...| United States| 1|
 United States| Croatia| 1|
 United States| Gibraltar| 1|
 United States| Singapore| 1|
 Moldova| United States| 1|
+--------------------+-------------------+-----+

## Repartition and Coalesce

In [ ]:
df.rdd.getNumPartitions() # 1

Out[124]: 1

In [ ]:
df.repartition(5).show()

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 United States| Denmark| 152|
 United States| Martinique| 43|
 United States| Saint Lucia| 136|
 Ireland| United States| 335|
 South Korea| United States| 1048|
 United States| Italy| 438|
 United States| Greece| 23|
Bonaire, Sint Eus...| United States| 58|
 France| United States| 935|
 United States| Cyprus| 1|
 United States| Montenegro| 1|
 United States| Austria| 63|
 Australia| United States| 329|
 United States| New Zealand| 74|
 United States| Suriname| 34|
 United States| Malaysia| 3|
 United States| Guyana| 63|
 Taiwan| United States| 266|
 Portugal| United States| 127|
 Bolivia| United States| 30|
+--------------------+-------------------+-----+
only showing top 20 rows

In [ ]:
df.repartition(col("DEST_COUNTRY_NAME")).show()

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 Moldova| United States| 1|
 Bolivia| United States| 30|
 Algeria| United States| 4|
Turks and Caicos ...| United States| 230|
 Pakistan| United States| 12|
 Marshall Islands| United States| 42|
 Suriname| United States| 1|
 Panama| United States| 510|
 New Zealand| United States| 111|
 Liberia| United States| 2|
 Ireland| United States| 335|
 Zambia| United States| 1|
 Malaysia| United States| 2|
 Japan| United States| 1548|
 French Polynesia| United States| 43|
 Singapore| United States| 3|
 Denmark| United States| 153|
 Spain| United States| 420|
 Bermuda| United States| 183|
 Kiribati| United States| 26|
+--------------------+-------------------+-----+
only showing top 20 rows

In [ ]:
df.repartition(5, col("DEST_COUNTRY_NAME")).show()

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 Moldova| United States| 1|
 Bolivia| United States| 30|
 Algeria| United States| 4|
Turks and Caicos ...| United States| 230|
 Pakistan| United States| 12|
 Marshall Islands| United States| 42|
 Suriname| United States| 1|
 Panama| United States| 510|
 New Zealand| United States| 111|
 Liberia| United States| 2|
 Ireland| United States| 335|
 Zambia| United States| 1|
 Malaysia| United States| 2|
 Japan| United States| 1548|
 French Polynesia| United States| 43|
 Singapore| United States| 3|
 Denmark| United States| 153|
 Spain| United States| 420|
 Bermuda| United States| 183|
 Kiribati| United States| 26|
+--------------------+-------------------+-----+
only showing top 20 rows

In [ ]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

Out[131]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

## Collecting Rows to the Driver

In [ ]:
collectDF = df.limit(10)
collectDF.take(5) # take works with an Integer count


Out[133]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [ ]:
collectDF.show() # this prints it out nicely


+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
 United States| Singapore| 1|
 United States| Grenada| 62|
 Costa Rica| United States| 588|
 Senegal| United States| 40|
 Moldova| United States| 1|
+-----------------+-------------------+-----+

In [ ]:
collectDF.show(5, False)


+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
United States |Romania |15 |
United States |Croatia |1 |
United States |Ireland |344 |
Egypt |United States |15 |
United States |India |62 |
+-----------------+-------------------+-----+
only showing top 5 rows

In [ ]:
collectDF.collect()

Out[138]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [ ]:
collectDF.toLocalIterator()

Out[139]: <generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x7f0657275820>

# Chapter 7 - Aggregations

In [ ]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("dbfs:/FileStore/regu-datasets/online_retail_dataset.csv")\
.coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")

## count

In [ ]:
from pyspark.sql.functions import count
df.select(count("StockCode")).show()

+----------------+
count(StockCode)|
+----------------+
 541909|
+----------------+

In [ ]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show()

+-------------------------+
count(DISTINCT StockCode)|
+-------------------------+
 4070|
+-------------------------+

In [ ]:
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode",0.1)).show()

+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 3364|
+--------------------------------+

##first and last

In [ ]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
first(StockCode)|last(StockCode)|
+----------------+---------------+
 85123A| 22138|
+----------------+---------------+

## min and max

In [ ]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
min(Quantity)|max(Quantity)|
+-------------+-------------+
 -80995| 80995|
+-------------+-------------+

## sum

In [ ]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show()

+-------------+
sum(Quantity)|
+-------------+
 5176450|
+-------------+

In [ ]:
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show()

/databricks/spark/python/pyspark/sql/functions.py:215: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
 warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)
+----------------------+
sum(DISTINCT Quantity)|
+----------------------+
 29310|
+----------------------+

In [ ]:
from pyspark.sql.functions import sum, count, avg, expr

df.select(
count("Quantity").alias("total_transactions"),
sum("Quantity").alias("total_purchases"),
avg("Quantity").alias("avg_purchases"),
expr("mean(Quantity)").alias("mean_purchases"))\
.selectExpr(
"total_purchases/total_transactions",
"avg_purchases",
"mean_purchases").show()

+--------------------------------------+----------------+----------------+
(total_purchases / total_transactions)| avg_purchases| mean_purchases|
+--------------------------------------+----------------+----------------+
 9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+

## Variance and Standard Deviation

In [ ]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
47559.30364660923| 47559.39140929892| 218.08095663447835| 218.08115785023455|
+-----------------+------------------+--------------------+---------------------+

##grouping

In [ ]:
df.groupBy("InvoiceNo", "CustomerId").count().show(3)

+---------+----------+-----+
InvoiceNo|CustomerId|count|
+---------+----------+-----+
 536366| 17850| 2|
 536367| 13047| 12|
 536369| 13047| 1|
+---------+----------+-----+
only showing top 3 rows

In [ ]:
from pyspark.sql.functions import count
df.groupBy("InvoiceNo").agg(
count("Quantity").alias("quan"),
expr("count(Quantity)")).show()

+---------+----+---------------+
InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
 536370| 20| 20|
 536380| 1| 1|
 536384| 13| 13|
 536387| 5| 5|
 536397| 2| 2|
 536405| 1| 1|
 536407| 2| 2|
 536463| 1| 1|
 536500| 15| 15|
 536522| 54| 54|
 536523| 12| 12|
 536536| 3| 3|
 536538| 31| 31|
 536542| 16| 16|
 536555| 2| 2|
 536561| 15| 15|
 536573| 4| 4|
 536579| 2| 2|
 536580| 6| 6|
 536582| 17| 17|
+---------+----+---------------+
only showing top 20 rows

In [ ]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
.show()

+---------+------------------+--------------------+
InvoiceNo| avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
 536370| 22.45| 8.935742834258381|
 536380| 24.0| 0.0|
 536384|14.615384615384615| 15.750645708563392|
 536387| 288.0| 117.57550765359255|
 536397| 30.0| 18.0|
 536405| 128.0| 0.0|
 536407| 6.0| 0.0|
 536463| 12.0| 0.0|
 536500| 6.8| 4.019950248448356|
 536522|1.5925925925925926| 1.6046058535136642|
 536523| 9.333333333333334| 7.487025815072067|
 536536|31.666666666666668| 34.373762603991366|
 536538| 4.709677419354839| 3.7173833008743054|
 536542| 24.5| 8.73212459828649|
 536555| 1.0| 0.0|
 536561| 9.333333333333334| 2.9814239699997196|
 536573| 34.0| 23.748684174075834|
 536579| 138.0| 78.0|
 536580| 11.0| 2.23606797749979|
 536582|10.823529411764707| 6.99777522727673|
+---------+------------------+--------------------+
only showing top 20 rows

## Window Functions

In [ ]:
from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window\
.partitionBy("CustomerId", "date")\
.orderBy(desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [ ]:
from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [ ]:
from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [ ]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")


Out[160]: DataFrame[key: string, value: string]

In [ ]:
from pyspark.sql.functions import col
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

+----------+----------+--------+------------+-----------------+-------------------+
CustomerId| date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
 12346|2011-01-18| 74215| 1| 1| 74215|
 12346|2011-01-18| -74215| 2| 2| 74215|
 12347|2010-12-07| 36| 1| 1| 36|
 12347|2010-12-07| 30| 2| 2| 36|
 12347|2010-12-07| 24| 3| 3| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 6| 17| 5| 36|
 12347|2010-12-07| 6| 17| 5| 36|
+----------+----------+--------+------------+-----------------+-------------------+
only showing top 20 rows

# Chapter 8 - Joins

In [ ]:
person = spark.createDataFrame([
(0, "Bill Chambers", 0, [100]),
(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
(500, "Vice President"),
(250, "PMC Member"),
(100, "Contributor")])\
.toDF("id", "status")

In [ ]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

## Inner Joins

In [ ]:
joinExpression = person["graduate_program"] == graduateProgram['id']

In [ ]:
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [ ]:
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

## Outer Joins"

In [ ]:
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [ ]:
joinType = "left_outer"

graduateProgram.join(person,joinExpression,joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 2|Masters| EECS|UC Berkeley|null| null| null| null|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+

In [ ]:
joinType ="right_outer"

person.join(graduateProgram , joinExpression , joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

## Semi Joins

In [ ]:
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
 id| degree| department| school|
+---+-------+--------------------+-----------+
 0|Masters|School of Informa...|UC Berkeley|
 1| Ph.D.| EECS|UC Berkeley|
+---+-------+--------------------+-----------+

In [ ]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([
(0, "Masters", "Duplicated Row", "Duplicated School")]))
gradProgram2.createOrReplaceTempView("gradProgram2")
gradProgram2.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------------+
 id| degree| department| school|
+---+-------+--------------------+-----------------+
 0|Masters|School of Informa...| UC Berkeley|
 1| Ph.D.| EECS| UC Berkeley|
 0|Masters| Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+

In [ ]:
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
 id| degree|department| school|
+---+-------+----------+-----------+
 2|Masters| EECS|UC Berkeley|
+---+-------+----------+-----------+

##Cross Joins

In [ ]:
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

In [ ]:
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 0| Bill Chambers| 0| [100]| 2|Masters| EECS|UC Berkeley|
 0| Bill Chambers| 0| [100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 2|Masters| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 2|Masters| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

##Handling Duplicate Column Names

In [ ]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")

In [ ]:
joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]

In [ ]:

person.join(gradProgramDupe, joinExpr).show()

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status|graduate_program| degree| department| school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+

In [ ]:
person.join(gradProgramDupe,"graduate_program").select("graduate_program").show()

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

In [ ]:
person.join(gradProgramDupe,joinExpr).drop(person["graduate_program"]).select("graduate_program").show()

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

## How Spark Performs Joins

In [ ]:
joinExpr = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpr).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#6666L], [id#6680L], Inner
 :- Sort [graduate_program#6666L ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(graduate_program#6666L, 5), ENSURE_REQUIREMENTS, [id=#7625]
 : +- Project [_1#6656L AS id#6664L, _2#6657 AS name#6665, _3#6658L AS graduate_program#6666L, _4#6659 AS spark_status#6667]
 : +- Filter isnotnull(_3#6658L)
 : +- Scan ExistingRDD[_1#6656L,_2#6657,_3#6658L,_4#6659]
 +- Sort [id#6680L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#6680L, 5), ENSURE_REQUIREMENTS, [id=#7626]
 +- Project [_1#6672L AS id#6680L, _2#6673 AS degree#6681, _3#6674 AS department#6682, _4#6675 AS school#6683]
 +- Filter isnotnull(_1#6672L)
 +- Scan ExistingRDD[_1#6672L,_2#6673,_3#6674,_4#6675]

In [ ]:
from pyspark.sql.functions import broadcast

person.join(broadcast(graduateProgram), joinExpr).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [graduate_program#6666L], [id#6680L], Inner, BuildRight, false
 :- Project [_1#6656L AS id#6664L, _2#6657 AS name#6665, _3#6658L AS graduate_program#6666L, _4#6659 AS spark_status#6667]
 : +- Filter isnotnull(_3#6658L)
 : +- Scan ExistingRDD[_1#6656L,_2#6657,_3#6658L,_4#6659]
 +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true]),false), [id=#7705]
 +- Project [_1#6672L AS id#6680L, _2#6673 AS degree#6681, _3#6674 AS department#6682, _4#6675 AS school#6683]
 +- Filter isnotnull(_1#6672L)
 +- Scan ExistingRDD[_1#6672L,_2#6673,_3#6674,_4#6675]

# Chapter 9 Data Sources

In [ ]:
read_data = spark.read.format("csv")\
.option("mode", "FAILFAST")\
.option("inferSchema", "true")\
.option("path", "dbfs:/FileStore/regu-datasets/housing.csv")\
.load()

In [ ]:
read_data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 _c0| _c1| _c2| _c3| _c4| _c5| _c6| _c7| _c8| _c9|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
 -122.25| 37.85| 52.0| 919.0| 213.0| 413.0| 193.0| 4.0368| 269700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 2535.0| 489.0| 1094.0| 514.0| 3.6591| 299200.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3104.0| 687.0| 1157.0| 647.0| 3.12| 241400.0| NEAR BAY|
 -122.26| 37.84| 42.0| 2555.0| 665.0| 1206.0| 595.0| 2.0804| 226700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3549.0| 707.0| 1551.0| 714.0| 3.6912| 261100.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2202.0| 434.0| 910.0| 402.0| 3.2031| 281500.0| NEAR BAY|
 -122.26| 37.85| 52.0| 3503.0| 752.0| 1504.0| 734.0| 3.2705| 241800.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2491.0| 474.0| 1098.0| 468.0| 3.075| 213500.0| NEAR BAY|
 -122.26| 37.84| 52.0| 696.0| 191.0| 345.0| 174.0| 2.6736| 191300.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2643.0| 626.0| 1212.0| 620.0| 1.9167| 159200.0| NEAR BAY|
 -122.26| 37.85| 50.0| 1120.0| 283.0| 697.0| 264.0| 2.125| 140000.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1966.0| 347.0| 793.0| 331.0| 2.775| 152500.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1228.0| 293.0| 648.0| 303.0| 2.1202| 155500.0| NEAR BAY|
 -122.26| 37.84| 50.0| 2239.0| 455.0| 990.0| 419.0| 1.9911| 158700.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 20 rows

In [ ]:
spark.read.format("csv")\
.option("header", "true")\
.option("mode", "FAILFAST")\
.option("inferSchema", "true")\
.load("dbfs:/FileStore/regu-datasets/housing.csv")

Out[213]: DataFrame[longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double, ocean_proximity: string]

In [ ]:
csvFile = spark.read.format("csv")\
.option("header", "true")\
.option("mode", "FAILFAST")\
.option("inferSchema", "true")\
.load("dbfs:/FileStore/regu-datasets/housing.csv")

In [ ]:
csvFile.write.format("csv").mode("overwrite").option("sep", "\t")\
.save("dbfs:/FileStore/regu-datasets/housing_tsv.tsv")

In [ ]:
spark.read.format("json").option("mode", "FAILFAST")\
.option("inferSchema", "true")\
.load("dbfs:/FileStore/2015_summary.json").show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
+-----------------+-------------------+-----+
only showing top 5 rows

In [ ]:
csvFile.write.format("json").mode("overwrite").save("dbfs:/FileStore/regu-datasets/my-json-file.json")

In [ ]:
csvFile.write.format("parquet").mode("overwrite").save("dbfs:/FileStore/regu-datasets/2010-summary.parquet")

In [ ]:
spark.read.format("parquet")\
.load("dbfs:/FileStore/regu-datasets/2010-summary.parquet").show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 5 rows

In [ ]:
csvFile.write.format("parquet").mode("overwrite")\
.save("dbfs:/FileStore/regu-datasets/my-parquet-file.parquet")

In [ ]:
csvFile = spark.read.format("json").option("mode", "FAILFAST")\
.option("inferSchema", "true")\
.load("dbfs:/FileStore/2015_summary.json")

In [ ]:
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")\
.save("dbfs:/FileStore/regu-datasets/partitioned-files.parquet")

In [ ]:
csvFile = spark.read.format("json").option("mode", "FAILFAST")\
.option("inferSchema", "true")\
.load("dbfs:/FileStore/2015_summary.json")
csvFile.limit(10).write.option("maxRecordsPerFile",500).mode("overwrite").partitionBy("DEST_COUNTRY_NAME")\
.save("dbfs:/FileStore/regu-datasets/maxRecordsPerFile-partitioned-files.parquet")

#Chapter 10 - Spark SQL

In [ ]:
spark.sql("SELECT 1 + 1").show()

+-------+
(1 + 1)|
+-------+
 2|
+-------+

In [ ]:
spark.read.json("dbfs:/FileStore/2015_summary.json")\
.createOrReplaceTempView("some_sql_view")

In [ ]:
spark.sql("""
SELECT DEST_COUNTRY_NAME , sum(count) FROM some_sql_view GROUP BY DEST_COUNTRY_NAME""")\
.where("DEST_COUNTRY_NAME LIKE 'S%'").where("`sum(count)` > 10")\
.count() 

Out[246]: 12

In [ ]:
%sql
CREATE TABLE flights1 (
DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path 'dbfs:/FileStore/2015_summary.json')

In [ ]:
%sql
CREATE TABLE flights_csv (
DEST_COUNTRY_NAME STRING,
ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
count LONG)
USING csv OPTIONS (header true, path 'dbfs:/FileStore/2015_summary.json')

In [ ]:
%sql
CREATE TABLE flights_from_select1 USING parquet AS SELECT * FROM flights1

In [ ]:
%sql
CREATE TABLE partitioned_flights1 USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights1 LIMIT 5

## Creating External Tables

In [ ]:
%sql
CREATE EXTERNAL TABLE hive_flights (
DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION 'dbfs:/data/flight-data-hive'

## Inserting into Tables

In [ ]:
%sql
INSERT INTO flights_from_select 
SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME , count FROM flights1 LIMIT 20

In [ ]:
%sql
INSERT INTO partitioned_flights
PARTITION (DEST_COUNTRY_NAME="UNITED STATES")
SELECT ORIGIN_COUNTRY_NAME,count  FROM flights
WHERE DEST_COUNTRY_NAME='UNITED STATES' LIMIT 12

##Describing and Refreshing Table Metadata

In [ ]:
%sql
DESCRIBE TABLE flights_csv

col_name,data_type,comment
DEST_COUNTRY_NAME,string,null
ORIGIN_COUNTRY_NAME,string,"remember, the US will be most prevalent"
count,bigint,null


In [ ]:
%sql 
SHOW PARTITIONS partitioned_flights

partition
DEST_COUNTRY_NAME=Afghanistan
DEST_COUNTRY_NAME=Egypt
DEST_COUNTRY_NAME=Equatorial Guinea
DEST_COUNTRY_NAME=Federated States of Micronesia
DEST_COUNTRY_NAME=Grenada
DEST_COUNTRY_NAME=India
DEST_COUNTRY_NAME=Ireland
DEST_COUNTRY_NAME=Marshall Islands
DEST_COUNTRY_NAME=Netherlands
DEST_COUNTRY_NAME=Romania


In [ ]:
%sql 
REFRESH table partitioned_flights

In [ ]:
%sql
MSCK REPAIR TABLE partitioned_flights

##Dropping Tables

In [ ]:
%sql
DROP TABLE IF EXISTS flights_csv;

##Caching Tables

In [ ]:
%sql 
CACHE TABLE flights

In [ ]:
%sql
UNCACHE TABLE FLIGHTS

##Views

In [ ]:
%sql

CREATE VIEW just_usa_view AS
SELECT * FROM flights WHERE dest_country_name = 'United States'

In [ ]:
%sql 
CREATE TEMP VIEW just_usa_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'

In [ ]:
%sql 
CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'

In [ ]:
%sql show tables

database,tableName,isTemporary
default,bucketedfiles,false
default,city_sm,false
default,flight_summary_2015,false
default,flights,false
default,flights1,false
default,flights_2,false
default,flights_3,false
default,flights_from_select,false
default,flights_from_select1,false
default,flights_new,false


In [ ]:
%sql CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'

In [ ]:
%sql 
SELECT * FROM just_usa_view_temp

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
United States,India,62
United States,Singapore,1
United States,Grenada,62
United States,Sint Maarten,325
United States,Marshall Islands,39
United States,Paraguay,6
United States,Gibraltar,1


In [ ]:
%sql DROP VIEW IF EXISTS just_usa_view;

##Databases

In [ ]:
%sql SHOW DATABASES

databaseName
default


In [ ]:
%sql
CREATE DATABASE some_db

In [ ]:
%sql show databases

databaseName
default
some_db


In [ ]:
%sql USE some_db

In [ ]:
%sql show tables

database,tableName,isTemporary
,dftable,true
,dftablelong,true
,dfwithdate,true
,flight_data_2015,true
,gradprogram2,true
,graduateprogram,true
,just_usa_view_temp,true
,person,true
,some_sql_view,true
,sparkstatus,true


In [ ]:
%sql SELECT * FROM default.flights

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


In [ ]:
%sql select current_database()

current_database()
some_db


In [ ]:
%sql use default

In [ ]:
%sql DROP DATABASE IF EXISTS some_db 

In [ ]:
%sql 
show databases

databaseName
default


##Select statements

In [ ]:
%sql 
SELECT * FROM flights

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


In [ ]:
%sql 
SELECT * FROM flights where DEST_COUNTRY_NAME = 'United States' order by ORIGIN_COUNTRY_NAME LIMIT 20

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Angola,13
United States,Anguilla,38
United States,Antigua and Barbuda,117
United States,Argentina,141
United States,Aruba,342
United States,Australia,258
United States,Austria,63
United States,Azerbaijan,21
United States,Bahrain,1
United States,Barbados,130


In [ ]:
%sql
SELECT
CASE WHEN DEST_COUNTRY_NAME = 'UNITED STATES' THEN 1
WHEN DEST_COUNTRY_NAME = 'Egypt' THEN 0
ELSE -1 END 
FROM partitioned_flights

CASE WHEN (DEST_COUNTRY_NAME = UNITED STATES) THEN 1 WHEN (DEST_COUNTRY_NAME = Egypt) THEN 0 ELSE -1 END
-1
0
-1
-1
-1
-1
-1
-1
-1
-1


## Structs and Lists

In [ ]:
%sql CREATE VIEW IF NOT EXISTS nested_data AS
SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights

In [ ]:
%sql SELECT * FROM nested_data

country,count
"List(United States, Romania)",15
"List(United States, Croatia)",1
"List(United States, Ireland)",344
"List(Egypt, United States)",15
"List(United States, India)",62
"List(United States, Singapore)",1
"List(United States, Grenada)",62
"List(Costa Rica, United States)",588
"List(Senegal, United States)",40
"List(Moldova, United States)",1


In [ ]:
%sql SELECT country.DEST_COUNTRY_NAME, count FROM nested_data

DEST_COUNTRY_NAME,count
United States,15
United States,1
United States,344
Egypt,15
United States,62
United States,1
United States,62
Costa Rica,588
Senegal,40
Moldova,1


In [ ]:
%sql SELECT country.*, count FROM nested_data

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


In [ ]:
%sql SELECT DEST_COUNTRY_NAME as new_name, collect_list(count) as flight_counts,
collect_set(ORIGIN_COUNTRY_NAME) as origin_set
FROM flights GROUP BY DEST_COUNTRY_NAME

new_name,flight_counts,origin_set
Algeria,List(4),List(United States)
Angola,List(15),List(United States)
Anguilla,List(41),List(United States)
Antigua and Barbuda,List(126),List(United States)
Argentina,List(180),List(United States)
Aruba,List(346),List(United States)
Australia,List(329),List(United States)
Austria,List(62),List(United States)
Azerbaijan,List(21),List(United States)
Bahrain,List(19),List(United States)


In [ ]:
%sql SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3) FROM flights

DEST_COUNTRY_NAME,"array(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
Egypt,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
Costa Rica,"List(1, 2, 3)"
Senegal,"List(1, 2, 3)"
Moldova,"List(1, 2, 3)"


In [ ]:
%sql SELECT DEST_COUNTRY_NAME as new_name, collect_list(count)[0]
FROM flights GROUP BY DEST_COUNTRY_NAME

new_name,collect_list(count)[0]
Algeria,4
Angola,15
Anguilla,41
Antigua and Barbuda,126
Argentina,180
Aruba,346
Australia,329
Austria,62
Azerbaijan,21
Bahrain,19


In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW flights_agg AS
SELECT DEST_COUNTRY_NAME, collect_list(count) as collected_counts
FROM flights GROUP BY DEST_COUNTRY_NAME

In [ ]:
%sql SELECT explode(collected_counts), DEST_COUNTRY_NAME FROM flights_agg

col,DEST_COUNTRY_NAME
4,Algeria
15,Angola
41,Anguilla
126,Antigua and Barbuda
180,Argentina
346,Aruba
329,Australia
62,Austria
21,Azerbaijan
19,Bahrain


## Functions

In [ ]:
%sql SHOW FUNCTIONS

function
!
!=
%
&
*
+
-
/
<
<=


In [ ]:
%sql SHOW SYSTEM FUNCTIONS

function
!
!=
%
&
*
+
-
/
<
<=


In [ ]:
%sql SHOW USER FUNCTIONS

function
getargument


In [ ]:
%sql SHOW FUNCTIONS LIKE "collect*";

function
collect_list
collect_set


##Subqueries

In [ ]:
%sql SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5

dest_country_name
United States
Canada
Mexico
United Kingdom
Japan


In [ ]:
%sql SELECT * FROM flights
WHERE origin_country_name IN (SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
Egypt,United States,15
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1
Guyana,United States,64
Malta,United States,1
Anguilla,United States,41
Bolivia,United States,30
Algeria,United States,4
Turks and Caicos Islands,United States,230


In [ ]:
%sql SELECT * FROM flights f1
WHERE EXISTS (SELECT 1 FROM flights f2
WHERE f1.dest_country_name = f2.origin_country_name)
AND EXISTS (SELECT 1 FROM flights f2
WHERE f2.dest_country_name = f1.origin_country_name)

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
United States,Sint Maarten,325


In [ ]:
%sql SELECT *, (SELECT max(count) FROM flights) AS maximum FROM flights

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count,maximum
United States,Romania,15,370002
United States,Croatia,1,370002
United States,Ireland,344,370002
Egypt,United States,15,370002
United States,India,62,370002
United States,Singapore,1,370002
United States,Grenada,62,370002
Costa Rica,United States,588,370002
Senegal,United States,40,370002
Moldova,United States,1,370002


## Setting Configuration Values in SQL

In [ ]:
%sql SET spark.sql.shuffle.partitions=20

key,value
spark.sql.shuffle.partitions,20


In [ ]:
%sql SET spark.sql.inMemoryColumnarStorage.compressed = true

key,value
spark.sql.inMemoryColumnarStorage.compressed,true
